ModComp Notebook!

In [116]:
import numpy as np
#det her er noget test kode jeg, Magnus Ejegod Bugge (den Tredje), 
#skrev for at prøve at lave det som et træ, men tror at Jonathans (den Første) kode er bedre

In [117]:
alpha0 = 50000
variance0 = 5000
deviation0 = np.sqrt(variance0)
alpha = 0
beta = 1
variance = 2500
deviation = np.sqrt(variance)
size = 1

In [147]:
class Node:
    def __init__(self,index,tree):
        
        self.id = tree[index,1]
        self.age_ch = tree[index,2]
        self.t = tree[index,3]
        self.species = tree[index,4]
        if tree[index,0] != 'NA':
            indexOfParent = np.where(tree[:,1]==tree[index,0])[0][0]
            self.parent = Node(indexOfParent,tree)
        else:
            self.parent = 'NA'
    def PrintTree(self):
        print(self.parent.id, self.id, self.age_ch, self.t, self.species)

    '''def insert(self, data):
        self.parent = data[0]
        if self.left == None:
            self.left = data[1]
        else:
            self.right = data[1]
            self.age_ch = data[2]
            self.t = data[3]
            self.species = data[4]
'''


In [149]:
z0 = np.random.normal(alpha0, deviation0, size)
tree = np.delete(np.loadtxt("./data/tree.csv", dtype=str, delimiter=","),0,0)

a = np.array([])
for i in range(0,204):
    b = Node(i,tree)
    a = np.append(a,b)

print(a[0].parent.children)





[]
